In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision import transforms
import time
from tqdm.autonotebook import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from inspect import getfullargspec,signature

import matplotlib.pyplot as plt
import numpy as np

from torch.utils.data import Dataset
from torchvision import datasets

import os
from PIL import ImageFile
from PIL import Image
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader

from robustness.datasets import Xray
ds = Xray('/tmp/train_ray')

%matplotlib inline
%config InlineBackend.figure_format = 'svg'


In [2]:
############################
class MnistResNet(nn.Module):
  def __init__(self, in_channels=1):
    super(MnistResNet, self).__init__()

    # Load a pretrained resnet model from torchvision.models in Pytorch
    self.model = models.resnet50(pretrained=False)

    # Change the input layer to take Grayscale image, instead of RGB images.  **************
    # Hence in_channels is set as 1 or 3 respectively
    # original definition of the first layer on the ResNet class
    # self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
    self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
    
    # Change the output layer to output 10 classes instead of 1000 classes
    num_ftrs = self.model.fc.in_features
    self.model.fc = nn.Linear(num_ftrs, 2)

  def forward(self, x):
    return self.model(x)


my_resnet = MnistResNet()

input = torch.randn((4,1,32,32))
output = my_resnet(input)
print(output.shape)
print(my_resnet)

torch.Size([4, 2])
MnistResNet(
  (model): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): S

In [3]:
        
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")
    
# load STD model 
model_std = MnistResNet().to(device)

# params you need to specify:
epochs = 150
batch_size = 4
model_std = MnistResNet()
model_state_dict = torch.load("/home/mugariya.farooq/Downloads/COVID-CT-master/model_backup/medical_transfer/covid_moco_covid.pt")
model_std.load_state_dict(model_state_dict,strict=False)


#model_std.to(device)

RuntimeError: version_ <= kMaxSupportedFileFormatVersion INTERNAL ASSERT FAILED at /pytorch/caffe2/serialize/inline_container.cc:132, please report a bug to PyTorch. Attempted to read a PyTorch file with version 3, but the maximum supported version for reading is 2. Your PyTorch installation may be too old. (init at /pytorch/caffe2/serialize/inline_container.cc:132)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x33 (0x7f38f012c193 in /home/mugariya.farooq/.conda/envs/mine/lib/python3.7/site-packages/torch/lib/libc10.so)
frame #1: caffe2::serialize::PyTorchStreamReader::init() + 0x1f5b (0x7f388c7ccc3b in /home/mugariya.farooq/.conda/envs/mine/lib/python3.7/site-packages/torch/lib/libtorch.so)
frame #2: caffe2::serialize::PyTorchStreamReader::PyTorchStreamReader(std::string const&) + 0x64 (0x7f388c7cde54 in /home/mugariya.farooq/.conda/envs/mine/lib/python3.7/site-packages/torch/lib/libtorch.so)
frame #3: <unknown function> + 0x6c5a36 (0x7f38cb1f6a36 in /home/mu

In [20]:
### Initiate ADV model 
from robustness.model_utils import make_and_restore_model
model_adv, _ = make_and_restore_model(arch='resnet50', dataset=ds,
              resume_path= '/home/mugariya.farooq/Downloads/COVID-CT-master/model_backup/medical_transfer/xrayAdv50/checkpoint.pt.latest') #'/tmp/35daedae-1b39-4941-ad08-8bd6459c1bd8/checkpoint.pt.best')
model_adv.eval()
pass

=> loading checkpoint '/home/mugariya.farooq/Downloads/COVID-CT-master/model_backup/medical_transfer/xrayAdv50/checkpoint.pt.latest'
=> loaded checkpoint '/home/mugariya.farooq/Downloads/COVID-CT-master/model_backup/medical_transfer/xrayAdv50/checkpoint.pt.latest' (epoch 150)


In [22]:
batchsize=4
normalize = transforms.Normalize(mean=[0.45271412, 0.45271412, 0.45271412],
                                     std=[0.33165374, 0.33165374, 0.33165374])

val_transformer = transforms.Compose([
#     transforms.Resize(224),
#     transforms.CenterCrop(224),
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    normalize
])

def read_txt(txt_path):
    with open(txt_path) as f:
        lines = f.readlines()
    txt_data = [line.strip() for line in lines]
    return txt_data



class CovidCTDataset(Dataset):
    def __init__(self, root_dir, txt_COVID, txt_NonCOVID, transform=None):
        """
        Args:
            txt_path (string): Path to the txt file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        File structure:
        - root_dir
            - CT_COVID
                - img1.png
                - img2.png
                - ......
            - CT_NonCOVID
                - img1.png
                - img2.png
                - ......
        """
        self.root_dir = root_dir
        self.txt_path = [txt_COVID,txt_NonCOVID]
        self.classes = ['CT_COVID', 'CT_NonCOVID']
        self.num_cls = len(self.classes)
        self.img_list = []
        for c in range(self.num_cls):
            cls_list = [[os.path.join(self.root_dir,self.classes[c],item), c] for item in read_txt(self.txt_path[c])]
            self.img_list += cls_list
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.img_list[idx][0]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        sample = {'img': image,
                  'label': int(self.img_list[idx][1])}
        return sample
    
testset = CovidCTDataset(root_dir='/home/mugariya.farooq/Downloads/COVID-CT-master/Images-processed/',
                              txt_COVID='/home/mugariya.farooq/Downloads/COVID-CT-master/Data-split/COVID/testCT_COVID.txt',
                              txt_NonCOVID='/home/mugariya.farooq/Downloads/COVID-CT-master/Data-split/NonCOVID/testCT_NonCOVID.txt',
                              transform= val_transformer)
test_loader = DataLoader(testset, batch_size=batchsize, drop_last=False, shuffle=False)

In [26]:
#only load the validation set
ATTACK_EPS = 0.5
ATTACK_STEPSIZE = 0.1
ATTACK_STEPS = 10
NUM_WORKERS = 8
BATCH_SIZE = 4

kwargs = {
    'constraint':'2', # use L2-PGD
    'eps': ATTACK_EPS, # L2 radius around original image
    'step_size': ATTACK_STEPSIZE,
    'iterations': ATTACK_STEPS,
    'do_tqdm': True,
}
#label.size()
from sklearn import metrics

# TensorDataset loaders
#_, test_loader = ds.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE) #make it bigger

test_enum = enumerate(test_loader)
#test_list = torch.empty(0)
#label_list = torch.empty(0)
acc_list = []

label_list = torch.empty(0)
test_list = torch.empty(0)
# Display image and label.
for _ in range(2) : #i, batch in enumerate(val_loader) :
    _,batch = next(enumerate(test_loader))
    im, label = batch["img"], batch["label"].float()
    
    test_list = torch.cat((test_list,im),dim=0)
    label_list =torch.cat((label_list,label),dim=0)
    

im, label = test_list.to(device),label_list.to(device)
label=label.long()

#for _ in range(len(test_loader)):
#    _, (im, label) = next(test_enum)  
#    im,label = im.to(device),label.to(device)
    
    #the model can generate adv by applying max l-2 loss
_, im_adv = model_adv(im, label, make_adv=True, **kwargs)
im_adv_transformed = transforms.Resize((32, 32)).forward(im_adv)
    
    
im_adv = im_adv.to(device='cpu')
im_adv_transformed = im_adv_transformed.to(device='cpu')
    
# Get predicted labels for adversarial examples using 
pred = model_std(im_adv_transformed)
label_pred = ch.argmax(pred, dim=1)
    
im,label = im.to(device='cpu'),label.to(device='cpu')
im_adv = im_adv.to(device='cpu')
im_adv_transformed = im_adv_transformed.to(device='cpu')
    
#Get accuracy
acc_list.append(metrics.accuracy_score(label ,label_pred.cpu()))
print("Accuracy:",metrics.accuracy_score(label ,label_pred.cpu()))
    
#test_list = torch.cat((test_list,im_adv_transformed.to(device='cpu')),dim=0)
#label_list = torch.cat((label_list,label),dim=0)

Current loss: 0.733473539352417: 100%|██████████| 10/10 [00:00<00:00, 30.92it/s]


AttributeError: 'Resize' object has no attribute 'forward'

In [22]:
print(torch.cuda.is_available())
torch.cuda.empty_cache()

True


In [19]:
## Generate adv examples 

print("Average over all batches ",   sum(acc_list) / len(acc_list))   

#im_adv_transformed = transforms.Resize((224, 224)).forward(im_adv)
#print(im_adv[0].size())
#print(im_adv_transformed[0].size())
# predict and measure the accuracy 

# Get predicted labels for adversarial examples using 
#pred = model_std(test_list)
#label_pred = ch.argmax(pred, dim=1)


ZeroDivisionError: division by zero

In [10]:
#norm = transforms.Compose([
#         transforms.ToTensor(),
#        transforms.Normalize((training_data.train_data.float().mean()/255,), 
#                            (training_data.train_data.float().std()/255,)),
#])

#from sklearn import metrics

#print("Accuracy:",metrics.accuracy_score(label_list.cpu() ,label_pred.cpu()))


Accuracy: 0.625
